In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path
import datetime
import bw2data as bd
import bw2io as bi
from shrecc.download import get_data
from shrecc.treatment import data_processing
from shrecc.database import create_database, filt_cutoff


# =======================================
# === Step 1: Download & Process Data ===
# =======================================

# === State ===
data = None

# === Default Data Path ===
PROJECT_ROOT = Path.cwd().parent
DEFAULT_DATA_PATH = PROJECT_ROOT / "data"

# === Widgets ===

data_path = widgets.Text(
    description="Data Path:",
    value=str(DEFAULT_DATA_PATH),
    layout=widgets.Layout(width="80%"),
)

year_dropdown = widgets.Dropdown(
    options=[str(y) for y in range(2011, datetime.datetime.now().year + 1)],
    description="Year:",
    value=str(2024),
)

download_button = widgets.Button(description="Download Data", button_style="info")

process_button = widgets.Button(description="Process Data", button_style="info")

# === Separate Output Boxes for each action ===

download_output = widgets.Output(
    layout=widgets.Layout(border="1px solid gray", height="60px", overflow="auto")
)

process_output = widgets.Output(
    layout=widgets.Layout(border="1px solid gray", height="60px", overflow="auto")
)

# === Event Handlers ===


def on_download_button_click(b):
    global data
    b.disabled = True
    b.button_style = ""

    with download_output:
        clear_output()
        try:
            year = int(year_dropdown.value)
            path = Path(data_path.value)

            print(f"📥 Downloading data for year {year} to {path}...")
            data = get_data(year=year, path_to_data=path)

            print("✅ Data downloaded successfully.")
        except Exception as e:
            print(f"❌ Error during data download: {e}")
        finally:
            b.disabled = False
            b.button_style = "info"


def on_process_button_click(b):
    b.disabled = True
    b.button_style = ""

    with process_output:
        clear_output()
        try:
            year = int(year_dropdown.value)
            path = data_path.value
            if data is None:
                print("⚠️ Please download data first.")
                return

            print("⚙️ Processing downloaded data...")
            data_processing(data_df=data, year=year, path_to_data=path)

            print("✅ Data processed successfully.")
        except Exception as e:
            print(f"❌ Error during data processing: {e}")
        finally:
            b.disabled = False
            b.button_style = "info"


download_button.on_click(on_download_button_click)
process_button.on_click(on_process_button_click)

# === Layout ===

step1_box = widgets.VBox(
    [
        widgets.HTML("<h3>Step 1: Download & Process Data</h3>"),
        year_dropdown,
        data_path,
        download_button,
        download_output,
        process_button,
        process_output,
    ]
)

# ================================================
# === Step 2: Set brightway Project & Database ===
# ================================================

# === Outputs ===
output_set_project = widgets.Output(
    layout=widgets.Layout(border="1px solid gray", height="60px", overflow="auto")
)

output_import_db = widgets.Output(
    layout=widgets.Layout(border="1px solid gray", height="60px", overflow="auto")
)

# === Widgets ===

# Project & DB inputs
project_name = widgets.Text(
    value="SHRECCei311", description="Project:", placeholder="Brightway project name"
)

set_project_button = widgets.Button(description="Set Project", button_style="warning")

# Ecoinvent import inputs
ecoinvent_version = widgets.Text(
    value="3.11",
    description="Version:",
    placeholder="e.g. 3.11",
    layout=widgets.Layout(width="150px", margin="0 20px 0 0"),
)

system_model = widgets.Dropdown(
    options=[
        "cutoff",
        "consequential",
        "allocation",
    ],  # TBC: do we need to limit to 'cutoff' only?
    value="cutoff",
    description="System model:",
    style={"description_width": "140px"},
    layout=widgets.Layout(width="300px"),
)

username = widgets.Text(
    value="",
    description="Username:",
    placeholder="Your ecoinvent username",
    layout=widgets.Layout(width="300px"),
)

password = widgets.Password(
    value="",
    description="Password:",
    placeholder="Your ecoinvent password",
    layout=widgets.Layout(width="300px"),
)

import_db_button = widgets.Button(
    description="Import Ecoinvent DB", button_style="danger"
)

# === Event Handlers ===


def on_set_project_click(b):
    b.disabled = True
    b.button_style = ""
    with output_set_project:
        clear_output()
        try:
            bd.projects.set_current(project_name.value)
            print(f"✅ Project set to '{project_name.value}'.")

            print("\n🔍 Current databases in project:")
            for db in bd.databases:
                print(f" - {db}")

        except Exception as e:
            print(f"❌ Error setting project: {e}")
        finally:
            b.disabled = False
            b.button_style = "warning"


def on_import_db_click(b):
    b.disabled = True
    b.button_style = ""
    with output_import_db:
        clear_output()
        try:
            b.ei_db_name = f"ecoinvent-{ecoinvent_version.value}-{system_model.value}"

            if b.ei_db_name in bd.databases:
                print(
                    f"ℹ️ '{b.ei_db_name}' database already present, no need to import."
                )
                return

            print(f"⏳ Importing {b.ei_db_name}, this may take some time...")
            bi.import_ecoinvent_release(
                version=ecoinvent_version.value,
                system_model=system_model.value,
                username=username.value,
                password=password.value,
            )
            print(f"✅ {b.ei_db_name} imported successfully.")
        except Exception as e:
            print(f"❌ Error importing database: {e}")
        finally:
            b.disabled = False
            b.button_style = "danger"


set_project_button.on_click(on_set_project_click)
import_db_button.on_click(on_import_db_click)

# === Layout ===

step2_box = widgets.VBox(
    [
        widgets.HTML("<h3>Step 2: Set Brightway Project & Database</h3>"),
        widgets.HTML("<b>Project set:</b>"),
        widgets.HBox([project_name, set_project_button]),
        output_set_project,
        widgets.HTML("<b>Ecoinvent Database Import (if missing):</b>"),
        widgets.HBox([ecoinvent_version, system_model]),
        username,
        password,
        import_db_button,
        output_import_db,
    ]
)

# ======================================
# === Step 3: Create SHRECC Database ===
# ======================================

output_freq = widgets.Output(
    layout=widgets.Layout(border="1px solid gray", height="60px", overflow="auto")
)
output_create_db = widgets.Output(
    layout=widgets.Layout(border="1px solid gray", height="60px", overflow="auto")
)

country_codes = [
    "AL",
    "AM",
    "AT",
    "AZ",
    "BA",
    "BE",
    "BG",
    "BY",
    "CH",
    "CY",
    "CZ",
    "DE",
    "DK",
    "EE",
    "ES",
    "FI",
    "FR",
    "GE",
    "GR",
    "HR",
    "HU",
    "IE",
    "IT",
    "LT",
    "LU",
    "LV",
    "MD",
    "ME",
    "MK",
    "MT",
    "NIE",
    "NL",
    "NO",
    "PL",
    "PT",
    "RO",
    "RS",
    "RU",
    "SE",
    "SK",
    "SI",
    "TR",
    "UA",
    "UK",
    "XK",
]

available_countries = widgets.SelectMultiple(
    options=country_codes,
    description="Available countries:",
    layout=widgets.Layout(width="250px", height="200px"),
    style={"description_width": "150px"},
)

selected_countries = widgets.SelectMultiple(
    options=[],
    description="Selected countries:",
    layout=widgets.Layout(width="250px", height="200px"),
    style={"description_width": "150px"},
)

add_button = widgets.Button(
    icon="arrow-right", tooltip="Add →", layout=widgets.Layout(width="40px")
)

remove_button = widgets.Button(
    icon="arrow-left", tooltip="← Remove", layout=widgets.Layout(width="40px")
)

# Calendar + time pickers
tz = datetime.timezone.utc

local_start = datetime.datetime(2024, 6, 1, 0, 0)  # local time
local_end = datetime.datetime(2024, 6, 30, 23, 0)  # local time
local_tz = datetime.datetime.now().astimezone().tzinfo

start_local = local_start.replace(tzinfo=local_tz)
end_local = local_end.replace(tzinfo=local_tz)

start_utc = start_local.astimezone(datetime.timezone.utc)
end_utc = end_local.astimezone(datetime.timezone.utc)

start_date_picker = widgets.DatetimePicker(
    description="Start:",
    disabled=False,
    value=start_utc,
)

end_date_picker = widgets.DatetimePicker(
    description="End:",
    disabled=False,
    value=end_utc,
)

# Frequency short code to label mapping
frequency_info = {
    "B": "business day frequency",
    "C": "custom business day frequency",
    "D": "calendar day frequency",
    "W": "weekly frequency",
    "ME": "month end frequency",
    "SME": "semi-month end (15th and end of month)",
    "BME": "business month end frequency",
    "CBME": "custom business month end frequency",
    "MS": "month start frequency",
    "SMS": "semi-month start (1st and 15th)",
    "BMS": "business month start frequency",
    "CBMS": "custom business month start frequency",
    "QE": "quarter end frequency",
    "BQE": "business quarter end frequency",
    "QS": "quarter start frequency",
    "BQS": "business quarter start frequency",
    "YE": "year end frequency",
    "BYE": "business year end frequency",
    "YS": "year start frequency",
    "BYS": "business year start frequency",
    "h": "hourly frequency",
    "bh": "business hour frequency",
    "cbh": "custom business hour frequency",
    "min": "minutely frequency",
    "s": "secondly frequency",
    "ms": "milliseconds",
    "us": "microseconds",
    "ns": "nanoseconds",
}

frequency_dropdown = widgets.Dropdown(
    options=list(frequency_info.keys()),
    value="h",
    description="Freq:",
    layout=widgets.Layout(width="150px"),
)

hour_start = widgets.BoundedIntText(
    value=10,
    min=0,
    max=23,
    description="Hour Start:",
    layout=widgets.Layout(width="150px"),
)

hour_end = widgets.BoundedIntText(
    value=14,
    min=0,
    max=23,
    description="Hour End:",
    layout=widgets.Layout(width="150px"),
)

cutoff_value = widgets.FloatText(value=1e-3, description="Cutoff:")

include_cutoff = widgets.Checkbox(value=True, description="Include Cutoff?")

db_name = widgets.Text(
    value="", placeholder="e.g. elec_june_2024_noon", description="DB name:"
)

create_db_button = widgets.Button(description="Create Database", button_style="success")

frequency_info_btn = widgets.Button(
    icon="info-circle",
    layout=widgets.Layout(width="30px"),
    tooltip="Show frequency descriptions",
)

# === Event Handlers ===


def add_selected_countries(b):
    new = list(available_countries.value)
    current = list(selected_countries.options)
    # Add unique ones only
    selected_countries.options = sorted(set(current + new))


def remove_selected_countries(b):
    remove = set(selected_countries.value)
    remaining = [c for c in selected_countries.options if c not in remove]
    selected_countries.options = remaining


add_button.on_click(add_selected_countries)
remove_button.on_click(remove_selected_countries)

country_select_box = widgets.HBox(
    [available_countries, widgets.VBox([add_button, remove_button]), selected_countries]
)


# Info icon (ℹ️) for frequency description
def show_frequency_info(b):
    with output_freq:
        clear_output()
        print("\nℹ️ Frequency Code Reference:")
        for code, desc in frequency_info.items():
            print(f"{code: <5} → {desc}")


def on_create_db_button_click(b):
    b.disabled = True
    b.button_style = ""
    with output_create_db:
        clear_output()
        try:
            start_dt_local = start_date_picker.value.astimezone(local_tz)
            end_dt_local = end_date_picker.value.astimezone(local_tz)

            start_dt_as_utc = start_dt_local.replace(tzinfo=datetime.timezone.utc)
            end_dt_as_utc = end_dt_local.replace(tzinfo=datetime.timezone.utc)

            print(
                f"Creating database for countries: {list(selected_countries.options)}"
            )
            print(f"Time period (UTC): {start_dt_as_utc} to {end_dt_as_utc}")
            print(f"Hours: {hour_start.value} to {hour_end.value}")
            print(f"Frequency: {frequency_dropdown.value}")
            print(f"Cutoff: {cutoff_value.value} (Include: {include_cutoff.value})")

            # Call SHRECC database creation function
            example = filt_cutoff(
                countries=list(selected_countries.options),
                general_range=[
                    start_utc.strftime("%Y-%m-%d %H:%M:%S"),
                    end_utc.strftime("%Y-%m-%d %H:%M:%S"),
                ],
                refined_range=[hour_start.value, hour_end.value],
                freq=frequency_dropdown.value,
                cutoff=cutoff_value.value,
                include_cutoff=include_cutoff.value,
                path_to_data=Path(data_path.value),
            )
            create_database(
                dataframe_filt=example,
                project_name=project_name.value,
                db_name=db_name.value,
                eidb_name=import_db_button.ei_db_name,
            )
            print("✅ SHRECC database created successfully.")
        except Exception as e:
            print(f"❌ Error creating database: {e}")

        finally:
            b.disabled = False
            b.button_style = "success"


frequency_info_btn.on_click(show_frequency_info)
create_db_button.on_click(on_create_db_button_click)

# === Layout ===

step3_box = widgets.VBox(
    [
        widgets.HTML("<h3>Step 3: Create SHRECC Database</h3>"),
        country_select_box,
        widgets.HBox([start_date_picker, end_date_picker]),
        widgets.HBox([hour_start, hour_end]),
        widgets.HBox([frequency_dropdown, frequency_info_btn]),
        output_freq,
        cutoff_value,
        include_cutoff,
        db_name,
        create_db_button,
        output_create_db,
    ]
)

# =======================
# === Final UI Layout ===
# =======================

# --- Top buttons to switch steps ---
step1_btn = widgets.Button(description="Step 1", button_style="info")
step2_btn = widgets.Button(description="Step 2", button_style="")
step3_btn = widgets.Button(description="Step 3", button_style="")

# Container for current step
step_container = widgets.VBox([step1_box])  # default show step 1


# --- Button click handlers ---
def show_step1(b):
    step_container.children = [step1_box]
    step1_btn.button_style = "info"
    step2_btn.button_style = ""
    step3_btn.button_style = ""


def show_step2(b):
    step_container.children = [step2_box]
    step1_btn.button_style = ""
    step2_btn.button_style = "warning"
    step3_btn.button_style = ""


def show_step3(b):
    step_container.children = [step3_box]
    step1_btn.button_style = ""
    step2_btn.button_style = ""
    step3_btn.button_style = "success"


# Connect buttons
step1_btn.on_click(show_step1)
step2_btn.on_click(show_step2)
step3_btn.on_click(show_step3)

# --- Top button layout ---
top_buttons = widgets.HBox(
    [step1_btn, step2_btn, step3_btn], layout=widgets.Layout(margin="0 0 10px 0")
)

# --- Final UI ---
ui = widgets.VBox([top_buttons, step_container])
display(ui)